In [26]:
import pandas as pd
import geopandas as gpd

This notebook aggregates the stops of the operators on a given grid.

In [27]:
if "snakemake" in locals():
    grid_path = snakemake.input[0]
    trajectories_paths = snakemake.input[1:]
    output_path = snakemake.output[0]
else:
    grid_path = "../../results/hexagons.gpkg"
    trajectories_paths = ["../../resources/operators/synthetic1/trajectories.csv"]
    output_path = "../../results/aggregation/stops.gpkg"

In [28]:
# Load grid
df_grid = gpd.read_file(grid_path)

In [29]:
# Count stops
df_stops = []

for path in trajectories_paths:
    df_operator = pd.read_csv(path)[["date", "longitude", "latitude"]]
    df_operator["geometry"] = gpd.points_from_xy(x = df_operator["longitude"], y = df_operator["latitude"])
    df_operator = gpd.GeoDataFrame(df_operator, crs = "EPSG:4326").to_crs(df_grid.crs)

    df_count = gpd.sjoin(df_operator, df_grid, predicate = "within")
    df_count = df_count.groupby("grid_id").size().reset_index(name = "stops")

    weight = 1.0 / len(df_operator["date"].unique())
    df_count["stops"] *= weight

    df_stops.append(df_count)

df_stops = pd.concat(df_stops).groupby("grid_id")["stops"].sum().reset_index()

In [30]:
# Merge with grid
df_grid = pd.merge(df_grid, df_stops, on = "grid_id", how = "left")
df_grid["stops"] = df_grid["stops"].fillna(0.0)

In [ ]:
# Output
df_grid[["stops", "geometry"]].to_crs("EPSG:4326").to_file(output_path)